[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/f_ivp/ivp.ipynb)

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from math import sin, cos, exp, sqrt
from scipy.integrate import ode
import sys

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Initial value problems

## Integrating the Rossler ODE system

The R\"ossler attractor is a chaotic ODE system with three coupled equations
$$
\begin{align*}
x' & = -y -z, \\
y' & = x+ay, \\
z' & = b+ z(x-c).
\end{align*}
$$
Consider solving this system using the trapezoid method,
$$
U^{n+1} - U^n = \frac{k\left[f(U^n,t_n) + f(U^{n+1},t_{n+1})\right]}{2}
$$
with $u(t) =
        (x(t),y(t),z(t))$. To evaluate $U^{n+1}$, we must find a root
of the equation
$$
F(U) = 2(U^n-U) + k \left[f(U^n,t_n) + f(U,t_{n+1})\right],
$$
where we will use Newton's method to perform the root-finding at each step.

### Problem setup

In [ ]:
# Constants in Rossler ODE system
a = 0.2
b = 0.2
c = 5.7

# Maximum number of Newton iterations
max_nsteps = 100

# Function to calculate derivatives x, y, and z in the Rossler attractor
def f(U, t):
    return np.array([-U[1]-U[2],
                     U[0]+a*U[1],
                     b+U[2]*(U[0]-c)])

# Function F to perform root-finding on, to compute the trapezoid update
# implicitly
def F(U, U_n, t, k):
    return 2*(U_n-U)+k*(f(U_n, t)+f(U, t+k))

# Jacobian of F, required for the Newton iteration
def J_F(U, t, k):
    return np.array([[-2, -k, -k],
                     [k, -2+a*k, 0],
                     [k*U[2], 0, -2+k*(U[0]-c)]])


# Initial condition, and step size
t = 0
U_n = np.array([-3., 0, 0])
k = 0.02

### Integrate ODE

In [ ]:
# Store the results for plotting
results = []

# Perform timesteps
print(t, U_n[0], U_n[1], U_n[2])
while t < 600:

    l = 0
    q = 0
    U = np.copy(U_n)

    # Perform Newton updates until two steps are below a tolerance
    while l < 2:

        # Calculate Newton iteration
        F_ = F(U, U_n, t, k)
        U -= np.linalg.solve(J_F(U, t, k), F_)

        # Check if the Newton iteration has converged
        if np.linalg.norm(F_) < 1e-12:
            l += 1
        else:
            l = 0

        # Check for too many iterations
        q += 1
        if q == max_nsteps:
            print("Too many Newton iterations")
            sys.exit()

    # Update solution using the result of the Newton iteration
    U_n = U
    t += k
    print(t, U_n[0], U_n[1], U_n[2], q)
    # Store results for plotting
    results.append([t, U_n[0], U_n[1], U_n[2]])

### Visualize

In [ ]:
# Extract time and x, y, z
results = np.array(results)
t = results[:, 0]
x = results[:, 1]
y = results[:, 2]
z = results[:, 3]

In [ ]:
# %matplotlib ipympl

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x, y, z, lw=0.5)
ax.scatter(x, y, z, c=t, cmap='viridis', s=0.1)

# Formatting
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.show()


In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig = plt.figure(figsize=(3, 3), dpi=50)
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
ax.set_xlim((x.min(), x.max()))
ax.set_ylim((y.min(), y.max()))
ax.set_zlim((z.min(), z.max()))

line, = ax.plot([], [], [], lw=0.5)
point, = ax.plot([], [], [], 'o', color='red')
time_text = ax.text2D(0.05, 0.95, '', transform=ax.transAxes)


def init():
    line.set_data([], [])
    line.set_3d_properties([])
    point.set_data([], [])
    point.set_3d_properties([])
    return line, point

def animate(frame):
    line.set_data(x[:frame], y[:frame])
    line.set_3d_properties(z[:frame])
    point.set_data([x[frame]], [y[frame]])
    point.set_3d_properties([z[frame]])
    return line, point

ani = FuncAnimation(fig, animate, frames=len(t), init_func=init, blit=True, interval=10)
plt.rcParams['animation.embed_limit'] = 1000

HTML(ani.to_jshtml())

## Stability of multistep methods

The code integrates three different multistep schemes:
- Stable:
    $$
    24U^{n+3} - 24U^{n+2} = k(9f(U^{n+3})+19f(U^{n+2}) - 5f(U^{n+1}) + f(U^n))
    $$
- Exponentially unstable:
    $$
    11U^{n+3}+27U^{n+2}-27U^{n+1}-11U^n = 3k(f(U^{n+3}) + 9f(U^{n+2}) + 9 f(U^{n+1}) + f(U^n))
    $$
- Algebraically unstable:
    $$
    U^{n+3} + U^{n+2} - U^{n+1} - U^{n} = 2k(f(U^{n+2})+ f(U^{n+1}))
    $$

### Set up functions

In [ ]:
# Function right hand side
def fun(t, u):
    return sin(t)-u

# Exact solution
def u_exact(t):
    return 0.5*(sin(t)-cos(t))+1.5*exp(-t)

# Integrate based on method
def integrate(method, fun=fun, u_exact=u_exact, output=False):
    # Total steps
    M = 101
    # Step size, and arrays for solution
    k = 10/(M-1)
    t = np.linspace(0, 10, M)
    f = np.empty((M))
    u = np.empty((M))

    # Populate the first few entries with the exact solution
    u[0:3] = [u_exact(z) for z in t[0:3]]
    f[0:3] = [fun(t[i], u[i]) for i in range(0, 3)]

    # Perform the steps of the method
    for n in range(0, M-3):
        # Stable
        if method == 0:
            u[n+3] = (u[n+2]+k*(9*sin(t[n+3])+19*f[n+2]-5*f[n+1]+f[n])/24) \
                / (1+9*k/24)
        # Exponentially unstable
        elif method == 1:
            u[n+3] = (-27*u[n+2]+27*u[n+1]+11*u[n]
                    + 3*k*(sin(t[n+3])+9*f[n+2]+9*f[n+1]+f[n])) \
                / (11+3*k)
        # Algebraically unstable
        else:
            u[n+3] = -u[n+2]+u[n+1]+u[n]+2*k*(f[n+2]+f[n+1])
        f[n+3] = fun(t[n+3], u[n+3])

    # Output the results
    if output:
        for n in range(0, M):
            ue = u_exact(t[n])
            print(t[n], u[n], f[n], ue, u[n]-ue)

    # Return the results
    ue = np.array([u_exact(tn) for tn in t])
    err = u - ue
    return t, u, f, ue, err

### Solve

In [ ]:
# Method:
# 0: stable
# 1: exponentially unstable
# 2: algebraically unstable

t0, u0, f0, ue0, err0 = integrate(method=0, output=False)
t1, u1, f1, ue1, err1 = integrate(method=1, output=False)
t2, u2, f2, ue2, err2 = integrate(method=2, output=False)

### Visualize

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=300)

# Plot numerical and exact solutions
ax.plot(t0, ue0, label='Exact solution', color='red', linestyle='--', lw=2)
ax.plot(t0, u0, label='Method 0', color='blue')
ax.plot(t1, u1, label='Method 1', color='green')
ax.plot(t2, u2, label='Method 2', color='orange')

# Formatting
ax.set_xlabel('$t$')
ax.set_ylabel('$u$')
ax.legend(loc='lower left')
ax.set_ylim(-1, 3)

plt.show()

## Stability calculation for other methods

This example computes the stability region for several multi-step methods in the BDF (backward differentiation formula) family:

- $11U^{n+3}-18U^{n+2} +9U^{n+1} - 2U^n = 6k f(U^{n+3})$
- $25U^{n+4}-48U^{n+3}+36U^{n+2} - 16U^{n+1} + 3U^n = 12k f(U^{n+4})$
- $137U^{n+5} - 300U^{n+4} + 300U^{n+3} - 200 U^{n+2} + 75 U^{n+1} -12 U^n = 60k f(U^{n+5})$

For each position $z$ in the complex plane, the program computes the maximum modulus of a root of $\pi(\zeta;z)$.

In [ ]:
# Polynomial coefficients
pol = np.array([11, -18, 9, -2.], dtype=complex); coeff = 6.
# pol=np.array([25,-48,36,-16,3],dtype=complex); coeff=12
# pol=np.array([137,-300,300,-200,75,-12],dtype=complex); coeff=60

# Computes the maximum magnitude of the root of the stability polynomial
# for lambda*h=x+y*i
def f(x, y):
    a = np.copy(pol)
    a[0] -= coeff*(x+1j*y)
    return np.max(np.abs(np.roots(a)))


# Plot results - find the 1 contour of max_r(|z_r(lambda*h)|) function
n = 129
xx = np.linspace(-20, 20, n)
yy = np.linspace(-20, 20, n)
X, Y = np.meshgrid(xx, yy)
pxy = np.empty((n, n))
for i in range(n):
    for j in range(n):
        pxy[j, i] = sqrt(f(xx[i], yy[j]))

fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=300)
plt.contourf(X, Y, pxy, 16, alpha=.75)
plt.contour(X, Y, pxy, levels=[1], colors='black')

# Formatting
ax.set_xlabel(r'Re$(z)$')
ax.set_ylabel(r'Im$(z)$')
ax.axis('equal')
plt.show()

## Stiff systems

### Explicit

In [ ]:
# Matrices
a = np.array([[998, 1998], [-999, -1999]])
i = np.identity(2)

# Initial conditions
ue = np.array([[1], [0]])
ui = np.array([[1], [0]])

# Starting time and timestep (currently chosen within the stability region of
# the explicit method)
t = 0
k = 0.02

# Store results
results_explicit = []

while t < 2:
    
    # Print solutions and exact solution
    ex1 = 2*exp(-t)-exp(-1000*t)
    ex2 = -exp(-t)+exp(-1000*t)
    print(t, ex1, ex2, ue[0, 0], ue[1, 0], ui[0, 0], ui[1, 0])
    results_explicit.append([t, ue[0, 0], ue[1, 0], ui[0, 0], ui[1, 0]])

    # Explicit step
    ue = ue+k*np.dot(a, ue)

    # Implicit step
    ui = np.linalg.solve(i-k*a, ui)
    t += k

### Implicit

In [ ]:
# Function
def f(t, y, arg1):
    return [998*y[0]+1998*y[1], -999*y[0]-1999*y[1]]


# Initial conditions
yinit = [1, 0]

# Set up stiff integrator and parameters
r = ode(f).set_integrator('zvode', method='bdf')
r.set_initial_value(yinit, 0).set_f_params(2.0).set_jac_params(2.0)
h = 0.05

# Store results
results_implicit = []

# Loop while the time is less than 2
while r.successful() and r.t < 2:
    
    r.integrate(r.t+h)

    # Print solution and comparison with exact answer
    ex1 = 2*exp(-r.t)-exp(-1000*r.t)
    ex2 = -exp(-r.t)+exp(-1000*r.t)
    print(r.t, float(r.y[0]), float(r.y[1]), ex1,
          ex2, float(r.y[0])-ex1, float(r.y[1])-ex2)
    results_implicit.append([r.t, float(r.y[0]), float(r.y[1]), ex1, ex2,
                             float(r.y[0])-ex1, float(r.y[1])-ex2])

### Plotting

In [ ]:
# Extract variables
results_explicit = np.array(results_explicit)
results_implicit = np.array(results_implicit)
time_explicit = results_explicit[:, 0]
u1_explicit = results_explicit[:, 1]
u2_explicit = results_explicit[:, 2]
time_implicit = results_implicit[:, 0]
u1_implicit = results_implicit[:, 1]
u2_implicit = results_implicit[:, 2]

# Plot the results
fig, axs = plt.subplots(2, 2, figsize=(6, 4), dpi=300)

# Explicit
axs[0,0].plot(time_explicit, u1_explicit, label='Explicit $u_1$', color='blue', linestyle=':')
axs[0,0].set_xlabel('$t$')
axs[0,0].set_ylabel('$u_1$')
axs[0,0].legend()
axs[0,1].plot(time_explicit, u2_explicit, label='Explicit $u_2$', color='blue', linestyle=':')
axs[0,1].set_xlabel('$t$')
axs[0,1].set_ylabel('$u_2$')
axs[0,1].legend()

# Implicit
axs[1,0].plot(time_implicit, u1_implicit, label='Implicit $u_1$', color='green', linestyle='--')
axs[1,0].set_xlabel('$t$')
axs[1,0].set_ylabel('$u_1$')
axs[1,0].legend()
axs[1,1].plot(time_implicit, u2_implicit, label='Implicit $u_2$', color='green', linestyle='--')
axs[1,1].set_xlabel('$t$')
axs[1,1].set_ylabel('$u_2$')
axs[1,1].legend()

plt.show()

## L-stability

It is preferable to have rapid decay of terms with large negative $\lambda$. This example demonstrates this for the trapezoid rule applied to the example stiff ODE.

### Trapezoid method

In [ ]:
# Matrices
a = np.array([[998, 1998], [-999, -1999]])
i = np.identity(2)

# Initial conditions
ut = np.array([[1], [0]])
ui = np.array([[1], [0]])

# Starting time and timestep (currently chosen within the stability region of
# the explicit method)
t = 0
k = 0.1

# Store results
results = []

while t < 10:

    # Print solutions and exact solution
    ex1 = 2*exp(-t)-exp(-1000*t)
    ex2 = -exp(-t)+exp(-1000*t)
    print(t, ex1, ex2, ut[0, 0], ut[1, 0], ui[0, 0], ui[1, 0])
    results.append([t, ut[0, 0], ut[1, 0], ui[0, 0], ui[1, 0]])

    # Trapezoid step
    tmp = ut+0.5*k*np.dot(a, ut)
    ut = np.linalg.solve(i-0.5*k*a, tmp)

    # Implicit step
    ui = np.linalg.solve(i-k*a, ui)
    t += k

### Plotting

In [ ]:
# Extract variables
results = np.array(results)
time = results[:, 0]
u1_explicit = results[:, 1]
u2_explicit = results[:, 2]
u1_implicit = results[:, 3]
u2_implicit = results[:, 4]

# Plot the results
fig, axs = plt.subplots(2, 2, figsize=(6, 4), dpi=300)

# Explicit
axs[0,0].plot(time, u1_explicit, label='Explicit $u_1$', color='blue', linestyle=':')
axs[0,0].set_xlabel('$t$')
axs[0,0].set_ylabel('$u_1$')
axs[0,0].legend()
axs[0,1].plot(time, u2_explicit, label='Explicit $u_2$', color='blue', linestyle=':')
axs[0,1].set_xlabel('$t$')
axs[0,1].set_ylabel('$u_2$')
axs[0,1].legend()

# Implicit
axs[1,0].plot(time, u1_implicit, label='Implicit $u_1$', color='green', linestyle='--')
axs[1,0].set_xlabel('$t$')
axs[1,0].set_ylabel('$u_1$')
axs[1,0].legend()
axs[1,1].plot(time, u2_implicit, label='Implicit $u_2$', color='green', linestyle='--')
axs[1,1].set_xlabel('$t$')
axs[1,1].set_ylabel('$u_2$')
axs[1,1].legend()

plt.show()